In [152]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import NMF

# Load the dataset
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
links_df = pd.read_csv('links.csv')

In [153]:
print("Movies DataFrame:")
print(movies_df.head())

print("\nRatings DataFrame:")
print(ratings_df.head())

print("\nLinks DataFrame:")
print(links_df.head())

Movies DataFrame:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Ratings DataFrame:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Links DataFrame:
   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        

In [154]:
# Create the user-item interaction matrix
user_item_matrix = ratings_df.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# # Fill NaN values with 0 (assuming no rating means no interaction)
# user_item_matrix.fillna(0, inplace=True)
# Display the shape of the matrix
print("Shape of the User-Item Interaction Matrix:", user_item_matrix.shape)

# Display the first few rows of the user-item interaction matrix
print(user_item_matrix.head())

Shape of the User-Item Interaction Matrix: (610, 9724)
movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           4.0     0.0     4.0     0.0     0.0     4.0     0.0     0.0   
2           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5           4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  9       10      ...  193565  193567  193571  193573  193579  193581  \
userId                   ...                                                   
1           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0    

In [155]:
# from sklearn.decomposition import NMF

# # Fill missing values with 0
# user_item_matrix_filled = user_item_matrix.fillna(0)

# # Set the number of latent features
# n_components = 10  # You can adjust this number

# # Create the NMF model
# nmf_model = NMF(n_components=n_components, random_state=42)

# # Fit the model to the user-item interaction matrix
# user_features = nmf_model.fit_transform(user_item_matrix_filled)
# item_features = nmf_model.components_

# # Reconstruct the user-item interaction matrix
# predicted_ratings = np.dot(user_features, item_features)

# # Convert to DataFrame for easier analysis
# predicted_ratings_df = pd.DataFrame(predicted_ratings, index=user_item_matrix.index, columns=user_item_matrix.columns)


# # Display the predicted ratings
# print(predicted_ratings_df.head())

# test 2
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF

# Set the number of latent features
n_latent_features = 20

# Initialize the NMF model
nmf_model = NMF(n_components=n_latent_features, init='random', random_state=42)

# Fit the model to the user-item interaction matrix
user_features = nmf_model.fit_transform(user_item_matrix)
item_features = nmf_model.components_

# Reconstruct the user-item interaction matrix
predicted_ratings = np.dot(user_features, item_features)

# Convert the predicted ratings to a DataFrame
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=user_item_matrix.index, columns=user_item_matrix.columns)

# Display the first few rows of the predicted ratings
print(predicted_ratings_df.head())


movieId    1         2         3         4         5         6         7       \
userId                                                                          
1        2.435419  1.048126  1.012087  0.040145  0.321829  1.619169  0.349166   
2        0.278372  0.126101  0.000000  0.000000  0.018414  0.096314  0.000000   
3        0.087198  0.039424  0.028666  0.000002  0.000017  0.069393  0.000479   
4        1.954110  0.357106  0.314678  0.065473  0.364281  1.004776  0.379547   
5        0.930768  0.994131  0.296295  0.087212  0.328511  0.498766  0.373269   

movieId    8         9         10      ...    193565    193567    193571  \
userId                                 ...                                 
1        0.069076  0.170971  1.515857  ...  0.000000  0.000000  0.000000   
2        0.002007  0.000000  0.030390  ...  0.003564  0.003055  0.004073   
3        0.000726  0.000263  0.071159  ...  0.000001  0.000001  0.000001   
4        0.021358  0.161114  0.248840  ...  0.000000

In [156]:
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np

# Extract the actual ratings where they are known
actual_ratings = user_item_matrix.values

# Extract the predicted ratings where they are known
predicted_ratings = predicted_ratings_df.values

known_ratings_mask = actual_ratings != 0

# actual_ratings = actual_ratings.fillna(0)
# predicted_ratings = predicted_ratings.fillna(0)

# Calculate RMSE
mse = mean_squared_error(actual_ratings[known_ratings_mask], predicted_ratings[known_ratings_mask])
# print(user_item_matrix_filled[known_ratings_mask])
rmse = np.sqrt(mse)
# Print the RMSE
print(f'RMSE: {rmse}')

RMSE: 2.4588660650487144


In [157]:
import pandas as pd
movies_df = pd.read_csv('movies.csv')

# Assuming `predicted_ratings_df` is a DataFrame of predicted ratings
user_id = 1  # Specify the user ID

# Step 1: Get the ratings of the selected user
user_ratings = user_item_matrix.loc[user_id]

# Step 2: Identify movies that the user has not rated
movies_not_rated = user_ratings[user_ratings == 0].index

# Step 3: Filter the predicted ratings to only include movies not rated by the user
predicted_for_user = predicted_ratings_df.loc[user_id, movies_not_rated]

# Step 4: Sort the predicted ratings in descending order and get the top 5
top_movies = predicted_for_user.sort_values(ascending=False).head(5).index

# test
recommended_movies = movies_df[ movies_df['movieId'].isin(top_movies)]

# print(top_movies.at[0, 'movieId'])

# Print the recommended movies
print("Top 5 recommended movies for user", user_id)
# print(top_movies)
print(recommended_movies[['movieId', 'title']])



Top 5 recommended movies for user 1
      movieId                                   title
507       589       Terminator 2: Judgment Day (1991)
793      1036                         Die Hard (1988)
902      1200                           Aliens (1986)
1057     1374  Star Trek II: The Wrath of Khan (1982)
2078     2762                 Sixth Sense, The (1999)


In [158]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


# Fill NaN values with 0 for similarity calculation
user_item_matrix_filled = user_item_matrix.fillna(0)

# Calculate the cosine similarity between users
similarity_matrix = cosine_similarity(user_item_matrix_filled)

# Convert the similarity matrix to a DataFrame for easier interpretation
similarity_df = pd.DataFrame(similarity_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)

# Display the similarity matrix
print("\nCosine Similarity Matrix:")
print(similarity_df)

# Specify the user ID to find similar users
user_id = 1  # Example user ID

# Display top 5 similar users (including the user itself)
top_similar_users = similarity_df[user_id].nlargest(6)  # Top 5 + user itself
print(f"\nTop 5 similar users to User {user_id}:")
print(top_similar_users)





Cosine Similarity Matrix:
userId       1         2         3         4         5         6         7    \
userId                                                                         
1       1.000000  0.027283  0.059720  0.194395  0.129080  0.128152  0.158744   
2       0.027283  1.000000  0.000000  0.003726  0.016614  0.025333  0.027585   
3       0.059720  0.000000  1.000000  0.002251  0.005020  0.003936  0.000000   
4       0.194395  0.003726  0.002251  1.000000  0.128659  0.088491  0.115120   
5       0.129080  0.016614  0.005020  0.128659  1.000000  0.300349  0.108342   
...          ...       ...       ...       ...       ...       ...       ...   
606     0.164191  0.028429  0.012993  0.200395  0.106435  0.102123  0.200035   
607     0.269389  0.012948  0.019247  0.131746  0.152866  0.162182  0.186114   
608     0.291097  0.046211  0.021128  0.149858  0.135535  0.178809  0.323541   
609     0.093572  0.027565  0.000000  0.032198  0.261232  0.214234  0.090840   
610     0.145

In [159]:
import pandas as pd

movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
links_df = pd.read_csv('links.csv')


def user_based_recommendation(user_id, user_similarity_df, user_item_matrix, n_recommendations=5):
    # Step 1: Get the most similar users
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:]  # Exclude the user themselves
    
    # Step 2: Get ratings from similar users
    similar_user_ratings = user_item_matrix.loc[similar_users]
    
    # Step 3: Compute weighted average of the ratings using similarity as weights
    weights = user_similarity_df[user_id][similar_users]
    weighted_ratings = similar_user_ratings.T.dot(weights)  # Dot product to get weighted ratings
    weighted_ratings /= weights.sum()  # Normalize by the sum of weights
    
    # Step 4: Exclude movies already rated by the user
    user_rated_movies = user_item_matrix.loc[user_id]
    weighted_ratings = weighted_ratings[user_rated_movies[user_rated_movies == 0].index]  # Keep only unrated movies
    
    # Step 5: Recommend the top N movies
    recommended_movies = weighted_ratings.sort_values(ascending=False).head(n_recommendations)
    
    return recommended_movies.index, recommended_movies.values  # Return movie IDs and scores

# Example usage
user_id = 1  # Specify the user ID for recommendations
recommended_movie_ids, recommended_scores = user_based_recommendation(user_id, similarity_df, user_item_matrix)

# Convert the recommended movie IDs to a DataFrame
recommended_movies_df = pd.DataFrame({
    'movieId': recommended_movie_ids,
    'score': recommended_scores
})

# Assuming you have a DataFrame `movies_df` with movie IDs and names
# Merging with movies_df to get movie names
merged_recommendations = recommended_movies_df.merge(movies_df, on='movieId', how='left')

# Assuming you have a DataFrame `links_df` with movie IDs and TMDB IDs
# Merging with links_df for the tmdbId
final_recommendations = merged_recommendations.merge(links_df, on='movieId', how='left')

# Display recommended movies with name, ID, and the link (tmdbId)
print(final_recommendations[['title', 'movieId', 'tmdbId']])


                                               title  movieId  tmdbId
0                   Shawshank Redemption, The (1994)      318   278.0
1                  Terminator 2: Judgment Day (1991)      589   280.0
2                              Godfather, The (1972)      858   238.0
3                            Sixth Sense, The (1999)     2762   745.0
4  Lord of the Rings: The Fellowship of the Ring,...     4993   120.0
